In [1]:
import nest_asyncio
import aiohttp
import asyncio
import nest_asyncio
import re
import time
import pandas as pd
import numpy as np
from dotenv import load_dotenv
import os
from os import getenv
from openai import OpenAI
import pathlib
import textwrap
# import google.generativeai as genai
import time
import requests
import ast
import json
from sklearn.metrics.pairwise import cosine_similarity
import re
from llms import chatGPT
from llms import gemini


x_chat = chatGPT()
x_gemini = gemini()

In [20]:
# df = pd.read_csv("../output/data/data_post_chat_gpt.csv",index_col=0)
df = pd.read_csv("../output/data/data.csv",index_col=0)
df = df[df["headquarters location"].str.contains("United States")]
# df = df[pd.to_datetime(df['founded date'])<='11/30/2022']

In [21]:
len(df)

8122

In [22]:
df = df.sample(3000, random_state=2)
#[df["organization name"].isin(["AI Mavericks","AI Teaches Me from BAM Software Development","AI Virtual Staging"])]
print("PRE", len(df[pd.to_datetime(df['founded date'])<='11/30/2022']))
print("POST", len(df[pd.to_datetime(df['founded date'])>='11/30/2022']))

PRE 2283
POST 717


In [23]:
generated_description_prompt = """
Tell me about ${company}'s product.

Next, rate your confidence in the description on a scale from 1 (not confident) to 10 (highly confident). Remember to follow the example format!

**EXAMPLES BELOW**

Company: Genmo
Website: https://www.genmo.ai/
Description: Genmo is a free tool that creates videos and images using artificial intelligence. Genmo can also generate creative text formats of text content, like poems, code, scripts, musical pieces, email, letters, etc.

Company: OnePane
Website: https://www.onepane.ai/
Description: Onepane is a company that offers an AI companion for enhanced DevOps & SRE efficiency. Onepane offers a GenAI solution providing unparalleled unified insights and control over your Cloud resources. Onepane helps with root cause analysis, cloud governance, and optimization strategies.

**YOUR TURN**

Company: $company
Website: $website
Current Product Description: $description
________________________
Description (in two sentences):
Confidence Interval:
Reasoning:
________________________
"""



In [24]:
filter_prompt = """
On a scale of 1-10, Is the goal of the product of $company ($website) to be used by a task & occupation in the US workforce? 

For each startup, provide
    + a specific rating (1-10) of how likely tool would be to replace part of a human workforce activity. 
    + If rating > 5, give ONET job title that the tool is automating.

**EXAMPLES BELOW**
Description: DecoverAI is an advanced research and eDiscovery platform tailored specifically for legal teams. It leverages cutting-edge AI technology to streamline and accelerate the discovery process, enabling lawyers to uncover critical insights and evidence efficiently.
Rating: 10
Reasoning (one sentence): Replacing research task for paralegal
ONET Task: Paralegal

Description: Flowfile is an AI-powered platform designed to streamline and automate repetitive tasks for real estate teams. Its features include document automation, lead qualification, and data management, freeing up valuable time for agents to focus on high-impact activities.
Rating: 10
Reasoning (one sentence): Replacing tedious tasks for real estate agents
ONET Task: Real Estate Agent

Description: AI Mavericks hosts high-profile events centered around AI and business growth, targeting founders, CEOs, and leadership teams seeking to leverage AI for strategic advantage. Their events provide an immersive platform for learning, networking, and gaining practical insights on integrating AI into business strategies.
Rating: 2
Reasoning (one sentence): Not replacing an occupation or task, just hosting events
ONET Task: N/A

**YOUR TURN**
Description of $company: $description
________________________
Rating: --
Reasoning: --
________________________
"""

In [25]:
parsed_description_prompt = """
Your role is to describe the jobs/tasks, industries, and customers that $company is targeting. Then, provide a confidence interval (1-10) on how sure you are about the response. 

For each startup, describe:
+ Tasks/Jobs being automated
+ Industry that tasks/jobs belong to
+ specific customers using the tool. 
    - DO NOT INCLUDE AN EXPLANATION

Remember to follow the example format!

**EXAMPLES BELOW**

Company: Petville
Website: petville.co/pricing/biz
Description: Petville Global is a B2B CRM SaaS platform that utilizes advanced technologies like AI/ML and neural net to streamline and expand operations for pet businesses and veterinary clinics both locally and globally. The platform offers deep data analytics and marketing tools, helping businesses save an average of 22% on CRM and vet tech costs.
Tasks/Jobs: Data analysis, Marketing automation, Appointment scheduling, Inventory management
Industry: Customer management
Customers: Veterinarians, Pet Businesses
Confidence Interval: 8
Reasoning: The description aligns well with the company's focus on CRM for pet-related businesses and includes specific tasks and industries that match the provided information.

Company: Thunder
Website: thundercompute.com
Description: Thunder is a decentralized, peer-to-peer cloud computing platform designed to democratize access to GPUs and address the persistent GPU shortage. It provides a solution for individuals and businesses seeking high-performance computing power, enabling them to leverage unused GPU resources from around the world.
Tasks/Jobs: GPU resource allocation, Access to unused GPUs, Distributed computing tasks
Industry: Cloud Computing
Customers: Developers, GPU Owners
Confidence Interval: 9
Reasoning: The platform's focus on GPU allocation and decentralized computing is clear, and the target customers and industry are well-defined and specific.

Company: NonprofitsHQ
Website: www.nonprofitshq.com
Description: NonprofitsHQ is a software suite designed for nonprofits that utilizes AI to automate tasks, manage operations, and improve efficiency, ultimately saving organizations time and resources.
Tasks/Jobs: Fundraising management, Donor relationship management, Grant writing, Event planning
Industry: Non-profit management
Customers: Non-profit organizations
Confidence Interval: 7
Reasoning: The description covers essential aspects of nonprofit management and identifies specific tasks and customers, but there could be more detail on the AI's role in these tasks.

**YOUR TURN**

Company: $company
Website: $website
Current Product Description: $description
________________________
Tasks/Jobs: (comma separated list of 4, short)
Industry: 
Customers: (comma separated list)
Confidence Interval: 
Reasoning: 
________________________
"""


In [26]:
#(follow ONET task statements for example on how to format a task: https://www.onetcenter.org/dictionary/20.1/excel/task_statements.html)

In [27]:
examples_prompt = """
Imagine you're explaining the product from $company to someone unfamiliar with it. Provide 3 concise examples of how this product could be used in a real-world scenario. Keep the descriptions general and avoid mentioning the company name.

For each example, describe:
+ Context: description of when a person would use the tool
+ Job Title and Task that the tool replaces
    - Job Title should be broad and not based on specific context!
    - Task should not use specific details from the context, just the general work activity
    **Here are 10 Example Job Titles and Tasks**
    ________________
    Job Title: Engineering teachers, postsecondary
    Task: Supervise undergraduate or graduate teaching, internship, and research work.
    
    Job Title: Instructional coordinators
    Task: Adapt instructional content or delivery methods for different levels or types of learners.
    
    Job Title: Technical writers
    Task: Edit, standardize, or make changes to material prepared by other writers or establishment personnel.
    
    Job Title: Nurse midwives
    Task: Plan, provide, or evaluate educational programs for nursing staff, health care teams, or the community.
    
    Job Title: Business teachers, postsecondary
    Task: Develop and maintain course web sites.
    
    Job Title: Airline pilots, copilots, and flight engineers
    Task: Coose routes, altitudes, and speeds that will provide the fastest, safest, and smoothest flights.
    
    Job Title: Electronic equipment installers and repairers, motor vehicles
    Task: Confer with customers to determine the nature of malfunctions.
    
    Job Title: Geography teachers, postsecondary
    Task: Maintain regularly scheduled office hours to advise and assist students.
    
    Job Title: Ophthalmologists, except pediatric
    Task: Document or evaluate patients' medical histories.
    
    Job Title: Special education teachers, elementary school
    Task: Collaborate with other teachers or administrators to develop, evaluate, or revise elementary school programs.
    ________________
+ Next, rate your confidence in example task on a scale from 1 (not confident) to 10 (highly confident).


**EXAMPLES BELOW**

Company: Blanc
Website: tryblanc.ai
Blanc is a compliance automation platform designed for fintech companies. It helps streamline regulatory compliance processes by providing a centralized hub for managing policies, monitoring activities, and generating reports.
________________
Context 1: A compliance officer at a fintech company uses Blanc to automate the process of creating and updating compliance policies, ensuring all documents are current and accessible to relevant team members.
Task 1: Verify that all firm and regulatory policies and procedures have been documented, implemented, and communicated.
Job Title 1: Compliance Officers

Context 2: A fintech company uses Blanc to generate automated compliance reports for regulatory audits, ensuring all necessary documentation is readily available and organized.
Task 2: Prepare reports of activities, evaluations, recommendations, or decisions.
Job Title 2: Compliance Officers

Context 3: A fintech company uses Blanc to monitor real-time transactions for potential compliance violations, triggering alerts and generating reports for further investigation.
Task 3: Identify compliance issues that require follow-up or investigation.
Job Title 3: Compliance Officers
________________

Company: Aether
Website: aetherenergie.com/
Aether Energy is an AI-driven platform designed to simplify the process of rooftop solar installation for businesses, providing comprehensive support from project planning and financing to installation and ongoing maintenance. This platform aims to streamline and optimize the entire solar energy journey for installers. 
________________
Context 1: A solar installer uses Aether to quickly create detailed project plans for rooftop solar installations, including system size, panel placement, and wiring diagrams. 
Task 1: Diagram layouts and locations for photovoltaic (PV) arrays and equipment, including existing building or site features.
Job Title 1: Solar Photovoltaic Installers

Context 2: A business owner leverages Aether to secure financing for their rooftop solar project, providing them with customized loan options and streamlined application processes. 
Task 2: Prepare solar installation project proposals, quotes, budgets, or schedules.
Job Title 2: Solar Photovoltaic Installers

Context 3: A solar installer uses Aether to manage the installation process, tracking materials, scheduling technicians, and coordinating with subcontractors, ensuring smooth project execution. 
Task 3:  Monitor work of contractors and subcontractors to ensure projects conform to plans, specifications, schedules, or budgets.
Job Title 3: Solar Energy Installation Managers
________________


Remember to follow the example format! Please include numbers!

**YOUR TURN**
Company:$company
Website: $website
Current Description: $description
$parsed_description
________________
**Example 1**
Context 1: 
Job Title 1: 
Task 1:
Confidence Interval 1:
Reasoning 1:

**Example 2**
Context 2: 
Job Title 2: 
Task 2:
Confidence Interval 2:
Reasoning 2:

**Example 3**
Context 3:
Job Title 3: 
Task 3:
Confidence Interval 3:
Reasoning 3:
________________
"""

In [28]:
# Apply nest_asyncio
nest_asyncio.apply()


class prompting():
    def __init__(self):
        self.results_df = pd.DataFrame(columns=["organization name", "value"])

    def set_current_results_df(self, results_df):
        self.results_df = results_df

    async def iterate(self, df, prompt_template, args, value, batch_size=10, start=0, end=False):
        if end == False:
            end = len(df)
        self.results_df = pd.DataFrame(columns=["organization name", value])
        if value in list(df.columns):
            if start != 0:
                self.results_df = pd.concat([df[["organization name", value]].iloc[:start], self.results_df], axis=0)
            df = df.drop(columns=[value])

        batch_prompts = []
        batch_indices = []

        for i, row in list(df.iterrows())[start:end]:
            name = row['organization name']
            website = row['website']
            prompt = prompt_template

            for arg in args:
                prompt = prompt.replace(f"${arg[0]}", row[arg[1]])

            batch_prompts.append((i, name, prompt))
            batch_indices.append(i)

            if len(batch_prompts) >= batch_size:
                await self.process_batch(batch_prompts)
                batch_prompts = []

        if batch_prompts:
            await self.process_batch(batch_prompts)
            self.results_df.to_csv("../output/current_results_df_prompting.csv")

        df = df.merge(self.results_df, on='organization name', how='left')
        return df

    async def process_batch(self, batch_prompts):
        async with aiohttp.ClientSession() as session:
            tasks = []
            for i, name, prompt in batch_prompts:
                tasks.append(self.fetch_result(session, i, name, prompt))
            await asyncio.gather(*tasks)

    async def fetch_result(self, session, i, name, prompt):
        failure_count = 0
        while True:
            try:
                print(f"******************************\nProcessing {i}: {name}")
                result = await x_gemini.ask(session, prompt)
                if result == "N/A": break  # explicit material

                text = re.sub(r"#|#\s+|_|\*", "", result).strip()

                self.results_df.loc[i] = [name, text]

                    
                break
            except Exception as e:
                print(failure_count)
                failure_count += 1
                if failure_count > 10:
                    break
                print(f"Error processing {i}, {name}: {e}")
                await asyncio.sleep(20)

prompting_class = prompting()


## Generated Description

In [29]:
args = [["company","organization name"], ["website","website"], ["description","description_all"]]

async def main(df):
    df = await prompting_class.iterate(df, generated_description_prompt, args, "generated_description_llm")
    df.to_csv('../output/df_with_generated_description.csv', index=False)
    return df

df = asyncio.run(main(df))

******************************
Processing 1537: Intelligent Racing
******************************
Processing 10443: Bright Feeds
******************************
Processing 21196: CTNEX
******************************
Processing 11667: Fathom
******************************
Processing 18931: Chronly
******************************
Processing 12125: ClickHouse
******************************
Processing 1831: Parasanti
******************************
Processing 4616: Boatrax
******************************
Processing 21211: MyJobAide LLC
******************************
Processing 9653: Talentprise
******************************
Processing 21444: TraceChain
******************************
Processing 10428: Ango AI
******************************
Processing 2101: Levogy
******************************
Processing 10356: Digital First AI
******************************
Processing 12691: Inergio.io
******************************
Processing 18377: Arkifi
******************************
Processing 1906: Wite

In [30]:
cols = ["generated_description", "generated_description_conf_interval", "generated_description_conf_interval_reasoning"]

def extract_data(text):
    if pd.isnull(text): return {}
    text = text.replace("\n"," ").replace("  "," ").replace("*","").replace("/10","").replace("_","").replace("#","")
    text = re.sub(r'\s?\([^)]*\)', '', text)
    pattern = r"^.*?\s*Description:?\s*(.*)Confidence Interval:\s*(\d+)\s*Reasoning:\s*(.*)$"
    match = re.search(pattern, text, re.DOTALL)
    
    if match:
        description = match.group(1).strip()
        confidence_interval = match.group(2).strip()
        reasoning = match.group(3).strip()
        
        result = {
            "generated_description": description,
            "generated_description_conf_interval": confidence_interval,
            "generated_description_conf_interval_reasoning": reasoning
        }
        return result
        
    else:
        print("FAILURE TO MATCH")
        return {}
        
for col in cols:
    if col in list(df.columns): df=df.drop(columns=[col])
        
results_df = pd.DataFrame(list(df.apply(lambda x: extract_data(x["generated_description_llm"]),axis=1)))
df = pd.concat([df, results_df],axis=1)
len(df)

FAILURE TO MATCH
FAILURE TO MATCH
FAILURE TO MATCH
FAILURE TO MATCH
FAILURE TO MATCH
FAILURE TO MATCH
FAILURE TO MATCH
FAILURE TO MATCH
FAILURE TO MATCH
FAILURE TO MATCH
FAILURE TO MATCH
FAILURE TO MATCH


3092

In [31]:
# df = pd.read_csv("../output/df_with_generated_description.csv", index_col=0).reset_index(drop=True)
# len(df)

In [32]:
df = df.dropna(subset=["generated_description"],axis=0)
df = df.drop_duplicates(subset=['organization name'], keep='first')
df = df.reset_index(drop=True)
len(df)

2940

## Filter Prompt

In [33]:
args = [["company","organization name"], ["website","website"], ["description","description_all"]]

async def main(df):
    df = await prompting_class.iterate(df, filter_prompt, args, "automation_rating_llm")
    df.to_csv('../output/df_with_filter.csv', index=False)
    return df

df = asyncio.run(main(df))

******************************
Processing 0: Intelligent Racing
******************************
Processing 1: Bright Feeds
******************************
Processing 2: CTNEX
******************************
Processing 3: Fathom
******************************
Processing 4: Chronly
******************************
Processing 5: ClickHouse
******************************
Processing 6: Parasanti
******************************
Processing 7: Boatrax
******************************
Processing 8: MyJobAide LLC
******************************
Processing 9: Talentprise
******************************
Processing 10: TraceChain
******************************
Processing 11: Ango AI
******************************
Processing 12: Levogy
******************************
Processing 13: Digital First AI
******************************
Processing 14: Inergio.io
******************************
Processing 15: Arkifi
******************************
Processing 16: WiteSand
******************************
Processing 17: Ains

In [64]:
cols = ["automation_rating", "automation_rating_reasoning"]

def extract_data(text):
    if pd.isnull(text): return {}
    text = re.sub(r'\s+', ' ', text).replace("\n"," ").replace("  "," ").replace("*","").replace("/10","").replace("_","").replace("#","")
    pattern = r"Rating: (\d+)\s+Reasoning:\s+(.*)"

    # Search for the pattern in the text
    match = re.search(pattern, text, re.DOTALL)
    if match:
        automation_rating = match.group(1).strip()
        automation_rating_reasoning = match.group(2).strip()

        result = {
            "automation_rating": automation_rating,
            "automation_rating_reasoning": automation_rating_reasoning
        }
        return result

    else:
        print(text)
        print("FAILURE TO MATCH")
        return {}
        
for col in cols:
    if col in list(df.columns): df=df.drop(columns=[col])
        
results_df = pd.DataFrame(list(df.apply(lambda x: extract_data(x["automation_rating_llm"]),axis=1)))
df = pd.concat([df, results_df],axis=1)


Exposit Systems, Inc. Description: — Stealth Rating: Impossible to assess Reasoning: Since Exposit Systems, Inc. is a stealth company, we have no information about their product or its intended use. Without knowing their product's functionality and target market, it's impossible to assess its potential impact on the workforce. ONET Task: N/A
FAILURE TO MATCH
It's impossible to give a rating or ONET task for ASL Multintel because it's operating in stealth mode. We have no information about their product or its intended use. Here's why this is problematic: Lack of Information: We don't know what kind of language learning tool they're developing. Is it software? A service? Stealth Mode: Stealth mode companies often don't publicly disclose details of their products or intended markets. To provide a rating, we'd need to know: What is the specific function of the tool? (e.g., language translation, sign language instruction, etc.) Who is the target audience? (e.g., students, professionals, ed

In [65]:
df = df.dropna(subset=["automation_rating"],axis=0)
df = df.drop_duplicates(subset=['organization name'], keep='first')
df = df.reset_index(drop=True)

In [66]:
# df = df[['organization name', 'full_description', 'industries',
#        'headquarters location', 'founded date', 'description', 'CB rank',
#        'headquarters region', 'postal code', 'estimated revenue',
#        'operating status', 'exit date', 'closed date', 'company type',
#        'website', 'twitter', 'facebook', 'linkedin', 'email', 'phone number',
#        'number of articles', 'hub tags', 'actively hiring', 'num of sub orgs',
#        'investor type', 'investor stage', 'number of founders', 'founders',
#        'number of employees', 'total funding amount',
#        'number of funding rounds', 'last funding amount', 'last funding type',
#        'description_all', 'industries_parsed', 'generated_description_llm',
#        'automation_rating_llm']]

In [68]:
for x in range(len(df))[:2]:
    if int(df.automation_rating[x]) <= 10:
        print(df.generated_description[x])
        print(df.automation_rating_llm[x])

Intelligent Racing offers professional training programs in artificial intelligence and machine learning, equipping individuals with the skills needed to succeed in this rapidly evolving field. Their curriculum covers a wide range of topics, from foundational concepts to advanced techniques, empowering learners to leverage AI for real-world applications.
Intelligent Racing Analysis

Rating: 2

Reasoning:  Intelligent Racing provides education and training, not a tool that directly automates a specific task or occupation.  The goal is to prepare people for AI-related roles, not replace them.

ONET Task: N/A
Bright Feeds is a company tackling food waste by converting it into animal feed using a combination of artificial intelligence and innovative drying technology. They aim to create a sustainable solution for food waste management, reducing landfill waste and providing valuable resources for animal feed production.
Bright Feeds Analysis

Rating: 7

Reasoning: While Bright Feeds doesn't

## Parsed Description

In [69]:
cols = ["parsed_description", "parsed_description_conf_interval", "parsed_description_conf_interval_reasoning", "Tasks/Jobs","Industry","Customers"]  
args = [["company","organization name"],["website","website"], ["description","generated_description"]]

async def main(df):
    df = await prompting_class.iterate(df, parsed_description_prompt, args, "parsed_description_llm")
    df.to_csv('../output/df_with_parsed_description.csv', index=False)
    return df

df = asyncio.run(main(df))




******************************
Processing 0: Intelligent Racing
******************************
Processing 1: Bright Feeds
******************************
Processing 2: CTNEX
******************************
Processing 3: Fathom
******************************
Processing 4: Chronly
******************************
Processing 5: ClickHouse
******************************
Processing 6: Parasanti
******************************
Processing 7: Boatrax
******************************
Processing 8: MyJobAide LLC
******************************
Processing 9: Talentprise
******************************
Processing 10: TraceChain
******************************
Processing 11: Ango AI
******************************
Processing 12: Levogy
******************************
Processing 13: Digital First AI
******************************
Processing 14: Inergio.io
******************************
Processing 15: Arkifi
******************************
Processing 16: WiteSand
******************************
Processing 17: Ains

In [70]:
def extract_data(text):
    if pd.isnull(text): return {}
    text = re.sub(r'\s+', ' ', text)  
    text = text.replace("_", "").replace("*", "").replace("#", "")
    text = re.sub(r'\s?\([^)]*\)', '', text)
    pattern = r".*?Tasks/Jobs:\s*(.*?)\s*Industry:\s*(.*?)\s*Customers:\s*(.*?)\s*Confidence Interval:\s*(.*?)\s*Reasoning:\s*(.*)"
    match = re.search(pattern, text, re.DOTALL)
    
    if match:
        tasks_jobs = match.group(1).strip()
        industry = match.group(2).strip()
        customers = match.group(3).strip()
        confidence_interval = match.group(4).strip()
        reasoning = match.group(5).strip()
        
        result = {
            "parsed_description": 
            "Tasks/Jobs: " + tasks_jobs + "\n" + "Industry: " + industry + "\n" + "Customers: "+ customers,
            "Tasks/Jobs": tasks_jobs,
            "Industry": industry,
            "Customers": customers,
            "parsed_description_conf_interval": confidence_interval,
            "parsed_description_conf_interval_reasoning": reasoning
        }
        
        return result
    else:
        print(text)
        print("FAILURE TO MATCH")
        return {}
        
for col in cols:
    if col in list(df.columns): df=df.drop(columns=[col])
        
results_df = pd.DataFrame(list(df.apply(lambda x: extract_data(x["parsed_description_llm"]),axis=1)))
df = pd.concat([df, results_df],axis=1)


In [71]:
len(df)

2928

In [72]:
df = df.dropna(subset=["parsed_description"],axis=0)
df = df.drop_duplicates(subset=['organization name'], keep='first')
df = df.reset_index(drop=True)
len(df)

2927

In [73]:
df.to_csv('../output/df_with_parsed_description.csv', index=False)

## Examples

In [74]:
cols = ["situation1", "situation1_conf_interval", "situation1_conf_interval_reasoning", "situation2", "situation2_conf_interval", "situation2_conf_interval_reasoning","situation3", "situation3_conf_interval", "situation3_conf_interval_reasoning",'Example1','Job1','Job1_title','Example2','Job2','Job2_title','Example3','Job3','Job3_title']
args = [["company","organization name"],["website","website"], ["description","generated_description"], ["parsed_description","parsed_description"]]


async def main(df):
    df = await prompting_class.iterate(df, examples_prompt, args, "examples_llm",1060)
    df.to_csv('../output/df_with_examples.csv', index=False)
    return df

df = asyncio.run(main(df))


******************************
Processing 0: Intelligent Racing
******************************
Processing 1: Bright Feeds
******************************
Processing 2: CTNEX
******************************
Processing 3: Fathom
******************************
Processing 4: Chronly
******************************
Processing 5: ClickHouse
******************************
Processing 6: Parasanti
******************************
Processing 7: Boatrax
******************************
Processing 8: MyJobAide LLC
******************************
Processing 9: Talentprise
******************************
Processing 10: TraceChain
******************************
Processing 11: Ango AI
******************************
Processing 12: Levogy
******************************
Processing 13: Digital First AI
******************************
Processing 14: Inergio.io
******************************
Processing 15: Arkifi
******************************
Processing 16: WiteSand
******************************
Processing 17: Ains

In [75]:

def extract_data(text):
    if pd.isnull(text): return {}
    # Normalize the text to ensure consistent whitespace and remove unwanted characters.
    text = re.sub(r'\s+', ' ', text).strip()
    text = text.replace("*", "").replace("/10", "").replace("_", "").replace("#", "").replace("- ","")
    text = re.sub(r'\(.*?\)', '', text)
    vals = {}
    i = 1
    while i <=3:
        # Regex pattern adjusted to handle fractions in confidence intervals like '9/10'
        pattern = rf"""
        Context\s+{i}:\s+(.*?)\s+
        Job Title\s+{i}:\s+(.*?)\s+
        Task\s+{i}:\s+(.*?)\s+
        Confidence\s+Interval\s+{i}:\s+(\d+(?:/\d+)?)\s+
        Reasoning\s+{i}:\s+(.*?)(?=\s*Context\s+{i + 1}:|$)"""
        pattern = pattern.replace("        ","").replace("\n","")
        match = re.search(pattern, text, re.DOTALL)
        if not match:
            print(text+"\n\n")
            print(f"No matches found for Example {i}")  # Debug if no examples are found
            return {}
            break

        example_text, job_title, task, confidence_interval, reasoning = match.groups()
        description = job_title + " that " + task.lower()
        example_key = f"Example{i}"
        vals[example_key] = example_text.strip()
        vals[f"Example_Task{i}"] = task.strip()
        vals[f"Example_Job_Title{i}"] = job_title.strip()
        vals[f"Example_Description{i}"] = description.strip()
        vals[f"situation{i}_conf_interval"] = confidence_interval.strip()
        vals[f"situation{i}_conf_interval_reasoning"] = reasoning.strip()

        i += 1  # Prepare to search for the next example
    return vals

for col in cols:
    if col in list(df.columns): df=df.drop(columns=[col])

results_df = pd.DataFrame(list(df.apply(lambda x: extract_data(x["examples_llm"]),axis=1)))
df = pd.concat([df, results_df],axis=1)


Apira Technologies Real-World Examples Example 1 Context 1: A social media influencer uses Apira's technology to create a photorealistic avatar for their Twitch streams. This avatar is used to interact with their audience in real-time, engaging viewers on a new level. Job Title 1: Social Media Manager Task 1: Engage with followers and cultivate a community around their content. Confidence Interval 1: 9 The use of a photorealistic avatar for audience interaction is directly related to the task of building a community and engagement. Reasoning 1: The avatar adds a personalized and interactive element to the streamer's content, which can increase viewer engagement and build a stronger sense of community. Example 2 Context 2: A virtual event organizer uses Apira's platform to create realistic avatars for attendees to interact with each other during virtual conferences and networking events. Job Title 2: Event Coordinator Task 2: Plan and manage the logistics of events. Confidence Interval 

In [76]:
df.to_csv('../output/df_with_examples.csv', index=False)

In [202]:
#RERUN STOPPED IN MIDDLE
# results_df = pd.read_csv("result.csv",index_col=0)
# df = pd.read_csv('../output/df_with_parsed_description.csv',index_col=0)
# prompting_class.set_current_results_df(results_df)
# df = df.merge(results_df, on='organization name', how='left')

In [25]:
# df = pd.read_csv('../output/results/df_with_examples.csv',index_col=0)


In [462]:
df = df[['organization name', 'full_description', 'industries',
       'headquarters location', 'founded date', 'description', 'CB rank',
       'headquarters region', 'postal code', 'estimated revenue',
       'operating status', 'exit date', 'closed date', 'company type',
       'website', 'twitter', 'facebook', 'linkedin', 'email', 'phone number',
       'number of articles', 'hub tags', 'actively hiring', 'num of sub orgs',
       'investor type', 'investor stage', 'number of founders', 'founders',
       'number of employees', 'total funding amount',
       'number of funding rounds', 'last funding amount', 'last funding type',
       'description_all', 'industries_parsed', 'generated_description_llm',
       'generated_description', 'generated_description_conf_interval',
       'generated_description_conf_interval_reasoning',
       'automation_rating_llm', 'automation_rating',
       'automation_rating_reasoning', 'parsed_description_llm',
       'parsed_description', 'Tasks/Jobs', 'Industry', 'Customers',
       'parsed_description_conf_interval',
       'parsed_description_conf_interval_reasoning', 'examples_llm']]

In [283]:
for i,x in df.iloc[:1].transpose().iterrows():
    print(i,x[0])

Unnamed: 0 1077
organization name AI Virtual Staging
full_description —
industries Artificial Intelligence (AI), Property Management, Real Estate, Real Estate Brokerage, Software
headquarters location San Francisco, California, United States
founded date 2023-02-01
description Real estate virtual staging using AI
CB rank —
headquarters region San Francisco Bay Area, West Coast, Western US
postal code —
estimated revenue —
operating status Active
exit date —
closed date —
company type For Profit
website www.virtualstaging.art
twitter —
facebook —
linkedin View on LinkedIn
email info@virtualstaging.art
phone number —
number of articles —
hub tags —
actively hiring —
num of sub orgs —
investor type —
investor stage —
number of founders —
founders —
number of employees —
total funding amount —
number of funding rounds —
last funding amount —
last funding type —
description_all Real estate virtual staging using AI —
industries_parsed ['Property Management', 'Real Estate', 'Real Estate Broke

In [478]:
df.automation_rating

AttributeError: 'DataFrame' object has no attribute 'automation_rating'